Pros:
    Very accurate and robust
    Does not suffer from overfitting
    Can do feature selection
Cons:
    Computational complexity
    Difficult to interpret?


In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



In [6]:
df = pd.read_csv('./Data Exploration/wdbc.csv')
df = df.drop(["Area", "AreaSE", "AreaWorst", "Perimeter", "PerimeterSE", "PerimeterWorst"], axis = 1)
encoder = LabelEncoder().fit(df["B/M"])
df['B/M'] = encoder.transform(df["B/M"])
not_data = ["ID", "B/M"]
label = df["B/M"]
feature = df.drop(not_data, axis = 1)
data = pd.concat([label, feature], axis=1)

No need to standardise as not sensitive to magnitude

In [7]:
x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.25, stratify=label)

In [8]:
# ROC ROC CURE, AUC CURVE

rfc = RandomForestClassifier(random_state=0, n_estimators=10)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)

print(accuracy_score(y_test, y_pred))
feature_scores = pd.Series(rfc.feature_importances_, index=x_train.columns).sort_values(ascending=False)
print(feature_scores)

0.972027972027972
RadiusWorst              0.298765
Concavity                0.270377
ConcavePointsWorst       0.089226
Compactness              0.086639
RadiusSE                 0.041497
ConcavityWorst           0.037414
TextureWorst             0.021240
Radius                   0.017632
CompactnessSE            0.014151
SymmetryWorst            0.014064
FractalDimension         0.014028
CompactnessWorst         0.013062
FractalDimensionSE       0.011705
SmoothnessWorst          0.010222
ConcavePoints            0.010179
Smoothness               0.009698
Symmetry                 0.008758
SmoothnessSE             0.008260
ConcavePointsSE          0.005820
Texture                  0.005295
FractalDimensionWorst    0.004146
ConcavitySE              0.003501
TextureSE                0.002812
SymmetrySE               0.001510
dtype: float64


In [9]:


cm = confusion_matrix(y_test, y_pred)

print('Confusion matrix\n\n', cm)

print(classification_report(y_test, y_pred))

Confusion matrix

 [[88  2]
 [ 2 51]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        90
           1       0.96      0.96      0.96        53

    accuracy                           0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143



In [ ]:
feature_scores = feature_scores.index.tolist()
important_features = feature_scores[:-10]
print(important_features)
feature = df[important_features]
print(feature)
x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.25, stratify=label)



In [ ]:
rfc = RandomForestClassifier(random_state=0, n_estimators=10)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)

print(accuracy_score(y_test, y_pred))
feature_scores = pd.Series(rfc.feature_importances_, index=x_train.columns).sort_values(ascending=False)

